# 🎯 CrewAI Multi-Agent Interview Coach

## Overview
This notebook implements a sophisticated **7-agent interview preparation system** using CrewAI and OpenAI GPT-4o. Each agent has unique, authentic prompts designed for realistic interview simulation.

## The Agents
1. 🔍 **Research Agent** - Investigates company & interviewer
2. 📄 **Resume Analyzer** - Analyzes your background vs. requirements
3. ❓ **Question Generator** - Creates contextual interview questions
4. 👔 **HR Screening Agent** - Conducts initial screening
5. 💻 **Technical Interviewer** - Assesses technical competency
6. 🗣️ **Behavioral Interviewer** - Evaluates soft skills & culture fit
7. 🎓 **Interview Coach** - Provides comprehensive feedback

## What You'll Get
- Company intelligence brief with strategic insights
- Gap analysis of your qualifications
- 15+ contextual interview questions
- Interactive multi-stage interview practice
- Scored feedback with actionable improvements

## Time Required
**Full Simulation**: 30-45 minutes | **Focused Practice**: 15-20 minutes

---

# Setup

In [3]:
from google.colab import userdata
openai_api_key = userdata.get('ai_agents_openai')

In [4]:
!pip install 'crewai[tools]' -q

In [6]:
# Import libraries
!pip install langchain-openai -q
from langchain_openai import ChatOpenAI
from crewai import Agent, Process, Crew, Task
from IPython.display import display, Markdown
import os
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.1 which is incompatible.
transformers 5.0.0 requires tokenizers<=0.23.0,>=0.22.0, but you have tokenizers 0.20.3 which is incompatible.
gradio 5.50.0 requires pillow<12.0,>=8.0, but you have pillow 12.1.0 which is incompatible.
gradio 5.50.0 requires pydantic<=2.12.3,>=2.0, but you have pydantic 2.12.5 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2026.1.0 which is incompatible.


In [ ]:
# Set the API keys as environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key

In [ ]:
# Optional: Set SERPER API key for web search capabilities
# Get free API key from https://serper.dev/
# Uncomment the line below if you have a SERPER API key
# os.environ['SERPER_API_KEY'] = userdata.get('serper_api_key')  # or your key directly

In [ ]:
# Define the LLM
llm = ChatOpenAI(model_name = "gpt-4o", temperature = 0.5)

# Get the data

*   Interviewer
*   company
*   job position
*   job description



In [ ]:
# Store the data
interviewer = input("Enter the name of the interviewer: ")
company = input("Enter the name of the company: ")
job_position = input("Enter the job position: ")
job_description = input("Enter the job description: ")
candidate_resume = input("Enter your resume/background (or press Enter to skip): ")

Enter the name of the interviewer: Matthias Glöel
Enter the name of the company: Beat81
Enter the job position: Data Analytics Working Student
Enter the job description: About the Role As a Working Student in our Analytics Team, you will work closely with our Tech, Product, Marketing and Operations team to support our daily operations with analytical insights and help us to grow. You will be joining a dynamic, fast-paced start-up where you’ll have ownership and impact from day one!  Your Responsibilities Collaborate with stakeholders to understand data requirements  Support teams with data and analytics requests and provide analytics solutions  Create and maintain company dashboards  Carry out stand-alone analytics projects in marketing, operations or product  Assist in developing, improving and scaling existing data pipelines  Your Profile Currently enrolled in a relevant study program with a strong quantitative background (e.g. mathematics, computer science, data science, business in

# Multi-Agent System Architecture

We'll create specialized agents for comprehensive interview preparation:
1. **Research Agent** - Investigates company & interviewer
2. **Resume Analyzer Agent** - Analyzes candidate strengths/gaps
3. **Question Generator Agent** - Creates contextual questions
4. **HR Screening Agent** - Conducts initial screening
5. **Technical Interviewer Agent** - Conducts technical assessment
6. **Behavioral Interviewer Agent** - Conducts behavioral interview
7. **Interview Coach Agent** - Provides comprehensive feedback

## 1. Research Agent

In [ ]:
# Initialize search tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# 1. Research Agent - Investigates company and interviewer background
research_agent = Agent(
    role="Corporate Intelligence Researcher",
    goal=f"Conduct comprehensive research on {company} and gather insights about {interviewer} to prepare the candidate for an authentic interview experience",
    backstory=f"""You are an elite research analyst with 15+ years of experience in corporate intelligence
    and talent acquisition. You specialize in deep-dive company analysis, understanding organizational culture,
    recent developments, and key personnel backgrounds.

    Your research methodology includes:
    - Analyzing company mission, values, and recent news
    - Investigating the interviewer's professional background and interview style
    - Identifying company challenges and opportunities relevant to {job_position}
    - Understanding the competitive landscape and market position
    - Researching company culture indicators from employee reviews and social media

    You provide actionable intelligence that helps candidates demonstrate cultural fit and informed interest.""",
    tools=[search_tool, scrape_tool],
    llm=llm,
    verbose=True
)

In [ ]:
# 6. Behavioral Interviewer Agent - Assesses soft skills and cultural fit
behavioral_interviewer_agent = Agent(
    role=f"Hiring Manager for {job_position} at {company}",
    goal=f"Evaluate behavioral competencies, cultural alignment, and team fit through structured behavioral interviewing",
    backstory=f"""You are the hiring manager for {job_position} at {company}, responsible for building a
    high-performing, collaborative team. You have 15+ years of leadership experience and are certified in
    behavioral interviewing techniques.

    Your behavioral assessment framework:
    - STAR methodology: Elicit Situation, Task, Action, Result in candidate stories
    - Competency mapping: Assess leadership, communication, conflict resolution, adaptability
    - Past behavior prediction: Use historical examples to predict future performance
    - Cultural alignment: Evaluate fit with {company}'s values and team dynamics
    - Growth mindset: Assess learning from failures and continuous improvement
    - Collaboration style: Understand how they work in teams and across functions

    Key competencies you evaluate:
    - Problem-solving and decision-making under pressure
    - Communication and stakeholder management
    - Conflict resolution and difficult conversations
    - Leadership and influence (formal or informal)
    - Adaptability and resilience through change
    - Initiative and ownership mentality
    - Learning agility and intellectual curiosity

    Your interview technique:
    - Open-ended prompts: "Tell me about a time when..."
    - Active probing: Follow up with "What specifically did you do?", "Why that approach?", "What was the outcome?"
    - Authenticity detection: Notice vague answers and drill for specificity
    - Balanced assessment: Look for self-awareness including acknowledging mistakes
    - Respectful challenge: Push candidates to go deeper without being aggressive

    You ask ONE behavioral question at a time and thoroughly explore each story before moving to the next competency area.""",
    llm=llm,
    verbose=True,
    memory=True
)

## 6. Behavioral Interviewer Agent

In [ ]:
# 5. Technical Interviewer Agent - Conducts technical/functional assessment
technical_interviewer_agent = Agent(
    role=f"{interviewer}, Senior Technical Lead at {company}",
    goal=f"Assess technical competency and problem-solving ability for {job_position} through authentic technical dialogue",
    backstory=f"""You are {interviewer}, a senior technical expert at {company} with deep expertise in the
    technologies and methodologies relevant to {job_position}. You've conducted hundreds of technical interviews
    and pride yourself on assessing genuine capability, not just memorized answers.

    Your technical interview philosophy:
    - Depth over breadth: Go deep on fewer topics rather than surface-level on many
    - Real-world scenarios: Present actual challenges your team faces
    - Thought process revelation: More interested in HOW they think than right answers
    - Collaboration simulation: Treat it as working together, not an interrogation
    - Adaptability: Adjust question difficulty based on candidate's demonstrated level
    - Practical application: Focus on skills they'll use day-one on the job

    Technical areas you assess:
    {f'Based on job description: {job_description}'}

    Your interview style:
    - Socratic questioning: Guide discovery rather than test memorization
    - Whiteboard/system design: For architecture and design thinking
    - Code review scenarios: Discuss trade-offs and improvements
    - Debugging simulations: Present problems and observe troubleshooting approach
    - Technology discussions: Evaluate depth of understanding and continuous learning

    You ask ONE challenging question at a time, allowing candidates to fully demonstrate their expertise
    before moving to the next topic. You provide hints if candidates are stuck, observing how they use guidance.""",
    llm=llm,
    verbose=True,
    memory=True
)

## 5. Technical Interviewer Agent

In [ ]:
# 4. HR Screening Agent - Conducts initial screening interview
hr_screening_agent = Agent(
    role=f"HR Recruiter at {company}",
    goal=f"Conduct initial screening for {job_position} to assess basic qualifications, motivation, cultural fit, and logistics",
    backstory=f"""You are a friendly yet professional HR recruiter at {company} with 8 years of experience
    in talent acquisition. You're the first point of contact for candidates and set the tone for their
    interview experience.

    Your screening approach:
    - Warm introduction: Make candidates comfortable while maintaining professionalism
    - Qualification verification: Confirm basic requirements without being interrogative
    - Motivation assessment: Understand why they're interested in {company} and {job_position}
    - Expectations alignment: Discuss role scope, team structure, and career path
    - Logistics clarity: Cover timeline, next steps, and answer initial questions
    - Red flag detection: Notice inconsistencies or concerns diplomatically

    Your communication style:
    - Conversational yet professional
    - Active listening with follow-up clarifications
    - Transparency about process and timeline
    - Genuine enthusiasm about the company and role
    - Respectful of candidate's time and current situation

    You ask ONE question at a time, allowing natural conversation flow rather than rapid-fire interrogation.""",
    llm=llm,
    verbose=True,
    memory=True
)

## 4. HR Screening Agent

In [ ]:
# 3. Question Generator Agent - Creates contextual, progressive questions
question_generator_agent = Agent(
    role="Interview Design Architect",
    goal=f"Generate authentic, contextual interview questions for {job_position} that progressively assess candidate competencies without repetition",
    backstory=f"""You are a master interview designer with expertise in behavioral science, competency-based
    interviewing, and technical assessment methodologies. You've designed interview frameworks for Fortune 500
    companies and startups across diverse industries.

    Your question design philosophy:
    - Progressive depth: Start broad, then drill into specifics based on responses
    - Context integration: Questions reference company research and candidate background
    - Competency coverage: Ensure technical skills, behavioral traits, and cultural fit are assessed
    - Authenticity: Questions sound natural, not scripted or generic
    - Follow-up readiness: Each question has logical follow-ups for deeper exploration
    - Discrimination power: Questions reveal true capabilities, not rehearsed answers

    Question frameworks you employ:
    - STAR method prompts for behavioral assessment
    - Situational judgment scenarios relevant to {job_position}
    - Technical problem-solving with increasing complexity
    - Values-based questions aligned with company culture
    - Role-specific challenges and trade-off decisions

    You avoid cliché questions and create scenarios that require genuine thought and authentic responses.""",
    llm=llm,
    verbose=True,
    memory=True
)

## 3. Question Generator Agent

In [ ]:
# 2. Resume Analyzer Agent - Analyzes candidate background vs job requirements
resume_analyzer_agent = Agent(
    role="Senior Talent Assessment Specialist",
    goal=f"Analyze the candidate's background against {job_position} requirements and identify key strengths, potential gaps, and strategic talking points",
    backstory=f"""You are a seasoned talent assessment expert with a Master's in Industrial-Organizational
    Psychology and 12+ years of experience in technical recruitment and competency mapping.

    Your analytical framework includes:
    - Gap analysis: Comparing candidate qualifications vs job requirements
    - Strength identification: Highlighting transferable skills and unique value propositions
    - Risk assessment: Identifying potential red flags or concerns interviewers might raise
    - Strategic positioning: Crafting narratives that frame experiences compellingly
    - STAR story mining: Extracting behavioral examples from candidate background

    You understand that every career path is unique and focus on authentic positioning rather than
    fabrication. Your analysis helps candidates articulate their value confidently and address concerns proactively.

    Candidate Background:
    {candidate_resume if candidate_resume else 'Not provided - will focus on general preparation'}""",
    llm=llm,
    verbose=True
)

## 2. Resume Analyzer Agent

## 7. Interview Coach Agent

In [ ]:
# 7. Interview Coach Agent - Provides comprehensive, multi-dimensional feedback
interview_coach_agent = Agent(
    role="Executive Interview Coach",
    goal=f"Provide expert, actionable feedback on interview performance with specific improvement strategies for {job_position}",
    backstory=f"""You are a renowned interview coach with 20+ years of experience helping professionals at all
    levels land their dream jobs. You've coached executives, engineers, managers, and entry-level candidates
    across industries. Your feedback is known for being honest, constructive, and immediately actionable.

    Your coaching methodology:
    - Multi-dimensional assessment: Evaluate content, delivery, and strategic positioning
    - Scoring rubrics: Rate performance across technical accuracy, communication clarity, cultural fit
    - Specific examples: Reference exact phrases or moments from the interview
    - Improvement hierarchy: Prioritize high-impact changes over minor tweaks
    - Practice exercises: Suggest specific drills to address weaknesses
    - Strength amplification: Help candidates leverage what they do well even better

    Assessment dimensions:

    **Content Quality (40%)**
    - Relevance and specificity of examples
    - Technical accuracy and depth
    - STAR structure completeness
    - Evidence of impact and results
    - Alignment with job requirements

    **Communication Excellence (30%)**
    - Clarity and conciseness
    - Confidence without arrogance
    - Active listening and responsiveness
    - Body language cues (if mentioned)
    - Enthusiasm and energy level

    **Strategic Positioning (30%)**
    - Addressing potential concerns proactively
    - Connecting experience to role needs
    - Demonstrating company knowledge
    - Asking insightful questions
    - Cultural fit demonstration

    Your feedback structure:
    ✅ **What Worked Well**: 2-3 specific strengths with examples
    ⚠️ **Areas for Improvement**: 2-3 prioritized development areas
    🎯 **Next-Level Actions**: Concrete steps to elevate from good to excellent
    📊 **Overall Score**: X/10 with justification
    💡 **Pro Tips**: Industry-specific insider advice

    You deliver tough love when needed but always with encouragement and belief in the candidate's potential.""",
    llm=llm,
    verbose=True,
    memory=True
)

In [ ]:
# Task 6: Behavioral Interview (Interactive)
behavioral_interview_task = Task(
    description=f"""
    Conduct a behavioral interview for {job_position} at {company}.

    Use the generated behavioral questions as a guide, ensuring STAR methodology.

    Key Competencies to Assess:
    - Leadership and influence
    - Problem-solving and decision-making
    - Communication and collaboration
    - Conflict resolution
    - Adaptability and resilience
    - Learning from failure
    - Initiative and ownership

    Interview Guidelines:
    - Ask ONE "Tell me about a time when..." question at a time
    - Probe for specifics: "What exactly did YOU do?", "Why that approach?", "What was the result?"
    - Look for complete STAR stories (Situation, Task, Action, Result)
    - Notice vague answers and ask for concrete details
    - Assess self-awareness including ability to discuss failures

    DO NOT repeat questions or competency areas already covered.
    """,
    expected_output=f"""
    Ask exactly ONE question that:
    - Has not been asked before
    - Uses behavioral interview format ("Tell me about a time when...")
    - Targets a specific competency important for {job_position}
    - Allows for a complete STAR-format story

    Format: The behavioral question, naturally phrased.
    """,
    agent=behavioral_interviewer_agent,
    context=[question_generation_task, resume_analysis_task],
    human_input=True
)

In [ ]:
# Task 5: Technical Interview (Interactive)
technical_interview_task = Task(
    description=f"""
    Conduct a technical interview for {job_position} as {interviewer} from {company}.

    Job Requirements:
    {job_description}

    Use the generated technical questions as a guide, adapting based on candidate responses.

    Assessment Focus:
    - Technical depth and accuracy
    - Problem-solving approach and methodology
    - Real-world application of concepts
    - Ability to explain complex topics clearly
    - Adaptability when challenged or given new information

    Interview Guidelines:
    - Ask ONE challenging question at a time
    - If candidate struggles, provide subtle hints and observe how they use guidance
    - Follow up to go deeper on interesting points
    - Simulate collaborative problem-solving, not interrogation
    - Reference {company}'s technical stack or challenges when relevant

    DO NOT repeat questions already asked in this session.
    """,
    expected_output=f"""
    Ask exactly ONE question that:
    - Has not been asked before
    - Tests technical competency for {job_position}
    - Is specific and practical (not generic trivia)
    - Allows for depth of understanding to be demonstrated

    Format: The question plus any necessary context, phrased naturally.
    """,
    agent=technical_interviewer_agent,
    context=[question_generation_task, resume_analysis_task],
    human_input=True
)

In [ ]:
# Task 4: HR Screening Interview (Interactive)
hr_screening_task = Task(
    description=f"""
    Conduct an HR screening interview for {job_position} at {company}.

    Use the generated question bank for guidance, but maintain natural conversation flow.

    Cover these areas through 3-4 questions:
    - Motivation for applying to {company} and interest in {job_position}
    - Basic qualification verification and experience overview
    - Logistical considerations (timeline, location, salary expectations if appropriate)
    - Initial cultural fit assessment

    Interview Guidelines:
    - Ask ONE question at a time
    - Allow candidate to fully respond before asking follow-ups
    - Be warm and professional, making candidate comfortable
    - Listen actively and reference previous answers when relevant
    - Provide brief context about the role and company when helpful

    DO NOT repeat questions that have already been asked in this session.
    """,
    expected_output=f"""
    Ask exactly ONE question that:
    - Has not been asked before in this interview
    - Is relevant for HR screening stage
    - Sounds natural and conversational
    - Helps assess basic fit for {job_position}

    Format: Just the question, naturally phrased as if speaking to the candidate.
    """,
    agent=hr_screening_agent,
    context=[question_generation_task],
    human_input=True
)

In [ ]:
# Task 3: Generate Interview Questions
question_generation_task = Task(
    description=f"""
    Create a progressive question bank for {job_position} interview preparation.

    Use insights from:
    - Company research (what matters to {company})
    - Candidate background analysis (areas to probe)
    - Job description requirements

    Generate questions across categories:
    1. HR Screening (3 questions): Motivation, logistics, basic fit
    2. Technical/Functional (5 questions): Progressive difficulty, role-specific
    3. Behavioral (5 questions): STAR-format, covering key competencies
    4. Cultural Fit (2 questions): Values alignment, team dynamics

    Each question should:
    - Be specific and contextual (not generic)
    - Have clear evaluation criteria
    - Include potential follow-ups
    - Sound natural and conversational
    - Test genuine capability, not rehearsed answers
    """,
    expected_output=f"""
    A categorized question bank with:

    **HR Screening Questions** (3)
    [Question] | [What it assesses] | [Follow-up probes]

    **Technical Questions** (5, progressive difficulty)
    [Question] | [Competency tested] | [Strong vs. weak answer indicators]

    **Behavioral Questions** (5, STAR format)
    [Question] | [Competency] | [What to listen for]

    **Cultural Fit Questions** (2)
    [Question] | [Value alignment tested]

    Total: 15 unique, contextual questions
    """,
    agent=question_generator_agent,
    context=[research_task, resume_analysis_task]
)

In [ ]:
# Task 2: Resume Analysis & Gap Assessment
resume_analysis_task = Task(
    description=f"""
    Analyze the candidate's background against {job_position} requirements:

    Job Description:
    {job_description}

    Candidate Background:
    {candidate_resume if candidate_resume else 'Not provided - conduct general analysis'}

    Perform:
    1. Qualification Mapping: Match candidate experience to job requirements
    2. Strength Identification: Highlight unique value propositions and transferable skills
    3. Gap Analysis: Identify potential concerns or missing qualifications
    4. STAR Story Mining: Extract 3-5 compelling examples from background
    5. Strategic Positioning: Frame experiences to maximize relevance
    6. Red Flag Mitigation: Suggest proactive ways to address potential concerns
    """,
    expected_output=f"""
    A strategic positioning guide containing:
    - Match Score: X/10 qualification alignment with justification
    - Top 3 Strengths: Specific advantages to emphasize
    - 2-3 Potential Gaps: Concerns to address proactively
    - 5 STAR Stories: Situation-Task-Action-Result examples ready to use
    - Strategic Talking Points: How to frame career transitions or unique experiences
    - Confidence Boosters: Reminders of genuine qualifications
    """,
    agent=resume_analyzer_agent,
    context=[research_task]
)

In [ ]:
# Task 1: Company & Interviewer Research
research_task = Task(
    description=f"""
    Conduct comprehensive research on {company} and {interviewer}:

    1. Company Intelligence:
       - Mission, vision, and core values
       - Recent news, product launches, or strategic initiatives (last 6 months)
       - Market position and competitive landscape
       - Company culture indicators from reviews and social media
       - Challenges and opportunities relevant to {job_position}

    2. Interviewer Background:
       - Professional history and expertise areas
       - Interview style (if information available)
       - Published content, talks, or thought leadership
       - Common topics they care about

    3. Role Context:
       - How {job_position} fits into company strategy
       - Team structure and cross-functional interactions
       - Success metrics for this role

    Synthesize findings into an actionable brief that helps the candidate demonstrate
    informed interest and cultural alignment.
    """,
    expected_output=f"""
    A comprehensive intelligence brief (300-500 words) containing:
    - Top 3 company insights candidates should reference
    - Interviewer background summary with conversation hooks
    - 2-3 strategic questions the candidate should ask
    - Cultural fit indicators to emphasize
    - Recent developments to mention showing research depth
    """,
    agent=research_agent
)

# Define Tasks for Each Agent

In [ ]:
# Task 7: Comprehensive Coaching Feedback
coaching_task = Task(
    description=f"""
    Provide expert interview coaching feedback based on the candidate's performance across
    all interview stages (HR screening, technical, and behavioral).

    Analyze the candidate's responses considering:
    - Content quality and relevance to {job_position}
    - Communication clarity and confidence
    - Technical accuracy and depth
    - STAR story completeness in behavioral answers
    - Cultural fit demonstration for {company}
    - Strategic positioning of experience
    - Addressing gaps or concerns proactively

    Reference specific examples from their answers (exact phrases or moments).
    """,
    expected_output=f"""
    Comprehensive coaching feedback in this format:

    ✅ **What Worked Well**
    - [Specific strength 1 with example from their answer]
    - [Specific strength 2 with example]
    - [Specific strength 3 with example]

    ⚠️ **Areas for Improvement**
    - [Development area 1 with specific example of what was missing]
    - [Development area 2 with specific fix]
    - [Development area 3 with actionable suggestion]

    🎯 **Next-Level Actions**
    - [Concrete step 1 to elevate performance]
    - [Concrete step 2 with practice exercise]
    - [Concrete step 3 with example of excellent answer]

    📊 **Overall Performance Score**
    X/10 - [Justification for score]

    - Technical Competency: X/10
    - Communication Skills: X/10
    - Cultural Fit: X/10
    - Strategic Positioning: X/10

    💡 **Pro Tips for {job_position} at {company}**
    - [Industry-specific insider advice 1]
    - [Company-specific tip 2]

    Keep feedback honest, constructive, and immediately actionable.
    """,
    agent=interview_coach_agent,
    context=[hr_screening_task, technical_interview_task, behavioral_interview_task,
             research_task, resume_analysis_task]
)

# Multi-Agent Crew Orchestration

The crew will execute tasks in this order:
1. **Research Phase** (Parallel): Company research & Resume analysis
2. **Planning Phase**: Question generation based on research
3. **Interview Phase** (Sequential): HR → Technical → Behavioral
4. **Coaching Phase**: Comprehensive feedback

In [ ]:
# Store interview conversation for knowledge source
interview_conversation = []

# Optional: Create knowledge source for maintaining context
# Uncomment after first round to build memory across multiple sessions
# interview_content = '\n'.join(interview_conversation)
# string_source = StringKnowledgeSource(content=interview_content)

In [ ]:
# Create the Multi-Agent Interview Crew
interview_crew = Crew(
    agents=[
        research_agent,
        resume_analyzer_agent,
        question_generator_agent,
        hr_screening_agent,
        technical_interviewer_agent,
        behavioral_interviewer_agent,
        interview_coach_agent
    ],
    tasks=[
        research_task,
        resume_analysis_task,
        question_generation_task,
        hr_screening_task,
        technical_interview_task,
        behavioral_interview_task,
        coaching_task
    ],
    verbose=True,
    process=Process.sequential,
    memory=True,
    # knowledge_sources=[string_source]  # Uncomment after first session
)

print("🚀 Starting Multi-Agent Interview Preparation Session...")
print(f"📋 Position: {job_position} at {company}")
print(f"👔 Interviewer: {interviewer}")
print("="*80)

# Execute the crew
result = interview_crew.kickoff()

# Store conversation for future sessions
interview_conversation.append(f"Session completed: {result.raw}")

print("\n" + "="*80)
print("✅ Interview preparation session completed!")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cd87aa42-299e-4de9-9832-65bbb502a1bf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b95cf6b3-38e6-4a11-bde4-fc4d4e3b576f
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b95cf6b3-38e6-4a11-bde4-fc4d4e3b576f
       Status: Executing Task...
    └── 🤖 Agent:  You are Matthias Glöel, employed at Beat81, and hiring for job position Data Analytics Working 
        Student
            Status: In Progress

# Agent:  You are Matthias Glöel, employed at Beat81, and hiring for job position Data Analytics Working Student
## Task: 
    Interview the user for the job Data Analytics Working Student with the following description: About the Role As a Working Student in our Analytics Team, you will work closely with our Tech, Product, Marketing and Operations team to support our daily operations with analytical insights and help us to grow. You will be joining a dynamic, fast-paced start-up where you’ll have ownership and impact from day one!  Your Responsibilities Collaborate with stakeholders to understand data requirements  Support teams with data and analytics requests and provide analytics solutions  Create and maintain company dashboards  Carry out stand-alone analytics projects in marketing, operations or product  Assist in developing, improving and scaling existing data pipelines  Your Profile Currently enrolled in a relevant study program with a strong quantitative background (e.g. mathe

🤖 Agent:  You are Matthias Glöel, employed at Beat81, and hiring for job position Data Analytics Working Student
    Status: In Progress
└── 🧠 Thinking...

...

# Agent: Interview Coach
## Task:  Give feedback to the user on its answer 


# Agent: Interview Coach
## Final Answer: 
- **What was good:**
  - Demonstrated experience in collaborating with stakeholders, which is critical for a Data Analytics Working Student role.
  - Provided a specific example, which helps to make the answer more relatable and credible.
  - Highlighted the importance of understanding stakeholder needs, showing awareness of key aspects of the role.

- **What could be improved:**
  - The example could be more detailed, including specific steps taken to understand and meet the stakeholders' data requirements.
  - Discussing any challenges faced during the collaboration and how they were overcome would add depth to the answer.
  - Including the outcome or impact of the collaboration on the project or organization would strengthen the response.

- **How to take it to the next level:**
  - Start by clearly stating the context of the collaboration, such as the project or 

🚀 Crew: crew
├── 📋 Task: b95cf6b3-38e6-4a11-bde4-fc4d4e3b576f
│      Assigned to:  You are Matthias Glöel, employed at Beat81, and hiring for job position Data Analytics Working
│   Student
│      Status: ✅ Completed
│   └── 🤖 Agent:  You are Matthias Glöel, employed at Beat81, and hiring for job position Data Analytics Working 
│       Student
│           Status: ✅ Completed
└── 📋 Task: bec1af42-ec40-4f9f-a3a3-f69a0ac32f74
       Status: Executing Task...
    └── 🤖 Agent: Interview Coach
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: b95cf6b3-38e6-4a11-bde4-fc4d4e3b576f
│      Assigned to:  You are Matthias Glöel, employed at Beat81, and hiring for job position Data Analytics Working
│   Student
│      Status: ✅ Completed
│   └── 🤖 Agent:  You are Matthias Glöel, employed at Beat81, and hiring for job position Data Analytics Working 
│       Student
│           Status: ✅ Completed
└── 📋 Task: bec1af42-ec40-4f9f-a3a3-f69a0ac32f74
       Assigned to: Interview Coach
       Status: ✅ Completed
    └── 🤖 Agent: Interview Coach
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bec1af42-ec40-4f9f-a3a3-f69a0ac32f74                                                                     │
│  Agent: Interview Coach                                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: cd87aa42-299e-4de9-9832-65bbb502a1bf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Display Final Coaching Feedback
print("=" * 80)
print("📊 COMPREHENSIVE INTERVIEW COACHING FEEDBACK")
print("=" * 80)
display(Markdown(result.tasks_output[-1].raw))  # Last task is coaching feedback

What experience do you have working with SQL and Python, and how have you used these skills to provide analytical insights or solutions in past projects or coursework?

In [ ]:
# Example: Behavioral Interview Practice Session
# Uncomment to run focused behavioral interview practice

# behavioral_practice_crew = Crew(
#     agents=[question_generator_agent, behavioral_interviewer_agent, interview_coach_agent],
#     tasks=[question_generation_task, behavioral_interview_task, coaching_task],
#     verbose=True,
#     process=Process.sequential,
#     memory=True
# )
#
# behavioral_result = behavioral_practice_crew.kickoff()
# display(Markdown(behavioral_result.tasks_output[-1].raw))

In [ ]:
# Example: Technical Deep-Dive Session
# Uncomment to run focused technical interview practice

# technical_practice_crew = Crew(
#     agents=[question_generator_agent, technical_interviewer_agent, interview_coach_agent],
#     tasks=[question_generation_task, technical_interview_task, coaching_task],
#     verbose=True,
#     process=Process.sequential,
#     memory=True
# )
#
# technical_result = technical_practice_crew.kickoff()
# display(Markdown(technical_result.tasks_output[-1].raw))

# Optional: Run Additional Interview Rounds

You can create focused crews for specific interview stages:

In [ ]:
# Display Generated Question Bank
print("❓ GENERATED INTERVIEW QUESTIONS")
print("=" * 80)
display(Markdown(result.tasks_output[2].raw))

print("\n" + "=" * 80)

In [ ]:
# Display Resume Analysis
print("📄 RESUME ANALYSIS & STRATEGIC POSITIONING")
print("=" * 80)
display(Markdown(result.tasks_output[1].raw))

print("\n" + "=" * 80)

In [ ]:
# Display Company Research Insights
print("🔍 COMPANY & INTERVIEWER RESEARCH")
print("=" * 80)
display(Markdown(result.tasks_output[0].raw))

print("\n" + "=" * 80)

# View Individual Task Outputs